# TensorFlow Custom Model
To define a custom model that works with tf.estimator, we need to use tf.estimator.Estimator. tf.estimator.LinearRegressor is actually a sub-class of tf.estimator.Estimator. Instead of sub-classing Estimator, we simply provide Estimator a function model_fn that tells tf.estimator how it can evaluate predictions, training steps, and loss:

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W * features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                    tf.assign_add(global_step, 1))
    # EstimatorSpec connects subgraphs we build to the
    # appropirate functionality.
    return tf.estimator.EstimatorSpec(mode=mode, predictions=y,
                                     loss=loss,train_op=train)



In [3]:
estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jf/_k92xf7x1n968vrldr_8vh7w0000gq/T/tmpe20rp9ej', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11bd75f98>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/jf/_k92xf7x1n968vrldr_8vh7w0000gq/T/tmpe20rp9ej/model.ckpt.
INFO:tensorflow:loss = 14.0762681948, step = 1
INFO:tensorflow:global_step/sec: 1262.63
INFO:tensorflow:loss = 0.187070850491, step = 101 (0.081 sec)
INFO:tensorflow:global_step/sec: 1497.25
INFO:tensorflow:loss = 0.0542616548427, step = 201 (0.067 sec)
INFO:tensorflow:global_step/sec: 1452.52
INFO:tensorflow:loss = 0.00355931388266, step = 301 (0.069 sec)
INFO:tensorflow:global_step/sec: 1558.39
INFO:tensorflow:loss = 0.000289035574781, step = 401 (0.064 sec)
INFO:tensorflow:global_step/sec: 1547.12
INFO:tensorflow:loss = 1.73999951022e-05, step = 501 (0.065 sec)
INFO:tensorflow:global_step/sec: 1506.05
INFO:tensorflow:loss = 1.71094899071e-06, step = 601 (0.066 sec)
INFO:tensorflow:global_step/sec: 1482.93
INFO:tensorflow:loss = 8.30022023471e-08, step = 701 (0.067 sec)
INFO:tensorflow:global_step/sec: 1439.08
INFO:tensorfl